# Write Siesta Run File

In [ ]:
def write_struct_fdf(geom, name: str, lattice_constant=None, unit='Ang',
fmt='.8f', path=None):
    # customize the struct.fdf file
    if not path:
        path = './opt'
    struct_file = name + '_STRUCT.fdf'

    if not lattice_constant:
        lattice_constant = 1.0 # Ang
    
    lat_con = round(lattice_constant, 8)

    cell_raw = geom.cell
    # nomalized cell
    cell = cell_raw/lat_con
    na = geom.na
    xyz = geom.xyz
    num_sp = len(geom.atoms.atom)

    with open(os.path.join(path, struct_file), 'w') as f:
        f.write(f"""LatticeConstant     {lat_con} {unit}
%block LatticeVectors""")
        for v in cell:
            f.write(str('\n'+f' {{:{fmt}}}'*3).format(
                *v))
        f.write(f"""
%endblock LatticeVectors

NumberOfAtoms   {na}
AtomicCoordinatesFormat {unit}
%block AtomicCoordinatesAndAtomicSpecies""")
        fmt_str = '\n' + f' {{:{fmt}}}'*3 + ' {} #  {}: {}'
        for ia, a, isp in geom.iter_species():
            f.write(fmt_str.format(*xyz[ia,:], isp+1, ia+1, a.tag))
        f.write(f"""
%endblock AtomicCoordinatesAndAtomicSpecies

NumberOfSpecies  {num_sp}
%block ChemicalSpeciesLabel""")
        for i, a in enumerate(geom.atoms.atom):
            f.write('\n {} {} {}'.format(i + 1, a.Z, a.tag))
        f.write(f"""
%endblock ChemicalSpeciesLabel
""")


In [ ]:
def create_siesta_runfile(geom, name:str, path=None, mpgrid=[20,1,1], lattice_constant=None,
 fmt='.8f', spin_polarized=True, spin_afm=True, write_wavefunctions=False, wavefunc_kpts:list=None, wfs_write_for_bands=False,
wfs_bands_range:list=None, denchar=False, optical_calc=False):
    # E and broadening should in eV unit, the program will convert them into Ry

    run_file = name + '_RUN.fdf'
    struct_file = name + '_STRUCT.fdf'
    if not path:
        path = './opt'

    mp1, mp2, mp3 = mpgrid

    xyz = geom.xyz

    with open(os.path.join(path, run_file), 'w') as f: #generating input file
        f.write(f"""%include {struct_file}
SystemName              {name}
SystemLabel             {name}
###########################################
#                 PAO
###########################################
PAO.BasisSize           DZP
PAO.BasisType           split
PAO.EnergyShift         150 meV
%block kgrid_Monkhorst_Pack
   {mp1}    0     0   0.0 
   0     {mp2}     0   0.0
   0     0     {mp3}   0.0
%endblock kgrid_Monkhorst_Pack
###########################################
#       Molecular Dynamics
###########################################
MD.VariableCell         T
MD.TypeOfRun            CG
MD.NumCGsteps           1000
MD.MaxCGDispl           0.05  Ang
MD.MaxForceTol          0.01 eV/Ang
MD.UseSaveXV            T
MD.UseSaveCG            T
###########################################
#         Density Matrix
###########################################
DM.UseSaveDM            T
MaxSCFIterations        500
DM.MixingWeight         0.05
DM.NumberPulay          4
DM.Tolerance            1.d-4
###########################################
#         Other parameters
###########################################
SolutionMethod          diagon
Diag.DivideAndConquer   False
MeshCutoff              400 Ry #converge
xc.functional           GGA #functional LDA vs GGA
xc.authors              PBE #functional PBE vs PBEsol etc.
ElectronicTemperature   300 K

#############################################
#         Band Structures
#############################################
BandLinesScale  pi/a
%block BandLines
  1         0.00000         0.00000         0.00000         \Gamma  
 100        1.00000         0.00000         0.00000         X
 100        2.00000         0.00000         0.00000         \Gamma  
%endblock BandLines

#############################################
#       SIETA Output Settings
#############################################

COOP.write              T
WriteDenchar            T
WriteMullikenPop        1
SaveRho                 T
WriteVoronoiPop         T
WriteEigenvalues        T
UseSaveData             T
SaveHS                  T
WriteCoorXmol           T
WriteCoorStep           T
WriteMDXmol             T
NetCharge               0
WriteBands              T

#######
#LDOS
#######
""")
        if write_wavefunctions:
            f.write("""
############################################
# Write Wavefunctions
############################################
WriteWaveFunctions      T
# the k points are in the scale of pi/a by default
%block WaveFuncKPoints
""")
            # selected k points and selected bands, give .selected.WFSX file
            for kpt in wavefunc_kpts:
                if not isinstance(kpt, list):
                    raise TypeError("The k points list should be a list of lists")
                f.write(" {} {} {} from {} to {} \n".format(*kpt, *wfs_bands_range))
            f.write("""%endblock WaveFuncKPoints\n""")
        
        # selected bands and all k points, give .bands.WFSX file
        if wfs_write_for_bands:
            f.write("""
WFS.Write.For.Bands     T
WFS.Band.Min            {}
WFS.Band.Max            {}
""".format(*wfs_bands_range)
)

        if denchar:
            denchar_file_name = name + ".denchar_input.fdf"
            f.write(f"%include {denchar_file_name}")

        if spin_polarized:
            f.write(f"""
##############################################
# Spin Settings
##############################################
Spin                    polarized
# this can only change the initial spin state, if read from DM, this parameter
# is useless. So, delete .DM file before change this parameter
DM.InitSpin.AF          {spin_afm}
""")
        if optical_calc:
            optcalc_file_name = name + '.optical_calc.fdf'
            f.write(f"""
#############################################
# Optical Calculation
#############################################
%include {optcalc_file_name}
""")


In [ ]:
def write_optical_calc_fdf(geom, name, path='./opt', 
energy_range=None, broaden=0,
scissor=0, num_bands=None,
kmesh=[20,1,1], polarization_type='unpolarized',
polarize_vector=[1,0,0]):
    filename = name + '.optical_calc.fdf'
    filepath = os.path.join(path, filename)
#     constant = 0.0734986176 # eV to Ry
    emin, emax = energy_range
#     emin *= constant
#     emax *= constant
#     broaden *= constant
#     scissor *= constant
    with open (filepath, 'w') as f:
        f.write(f"""
OpticalCalculation      True
Optical.Energy.Minumum  {emin} eV
Optical.Energy.Maximum  {emax} eV
Optical.Broaden         {broaden} eV
Optical.Scissor         {scissor} eV""")
        if num_bands:
            f.write(f"""
Optical.NumberOfBands   {num_bands}
""")
        f.write("""
%block Optical.Mesh
""")
        f.write(' {} {} {}'.format(*kmesh))
        f.write(f"""
%endblock Optical.Mesh
Optical.PolarizationType   {polarization_type}
%block Optical.Vector""")
        f.write('\n {:.1f} {:.1f} {:.1f}'.format(*polarize_vector))
        f.write("""
%endblock Optical.Vector""")



In [ ]:
def write_denchar_input_file(geom, name, path='./opt', type_of_run='3D', 
plot_charge=True, plot_wavefunctions=True, coor_units='Ang'):
    filename = name + '.denchar_input.fdf'
    filepath = os.path.join(path, filename)
    num_sp = len(geom.atoms.atom)
    xyz = geom.xyz
    xmax, ymax, zmax = np.max(xyz, axis=0)-geom.center() #*1.88973
    xmax += 20
    ymax += 5
    zmax += 3
    xmin, ymin, zmin = np.min(xyz, axis=0)-geom.center() #*1.88973
    xmin -= 20
    ymin -= 5
    zmin -= 3
    center = geom.center()
    xaxis = center + np.array([3,0,0])

    with open (filepath, 'w') as f:
        f.write(f"""
SystemLabel             {name}
NumberOfSpecies         {num_sp}
%block ChemicalSpeciesLabel""")

        for i, a in enumerate(geom.atoms.atom):
            f.write('\n {} {} {}'.format(i + 1, a.Z, a.tag))
        f.write(f"""
%endblock ChemicalSpeciesLabel

Denchar.TypeOfRun       {type_of_run}
Denchar.PlotCharge      {plot_charge}
Denchar.PlotWaveFunctions   {plot_wavefunctions}

Denchar.CoorUnits       {coor_units}""")
        f.write("""
%block Denchar.PlaneOrigin
 {} {} {}
%endblock Denchar.PlaneOrigin
""".format(*center))
        f.write("""
%block Denchar.X-Axis
 {} {} {}
%endblock Denchar.X-Axis
""".format(*xaxis))
        f.write("""
Denchar.MaxX            {} Ang
Denchar.MaxY            {} Ang
Denchar.MaxZ            {} Ang
Denchar.MinX            {} Ang
Denchar.MinY            {} Ang
Denchar.MinZ            {} Ang
""".format(xmax, ymax, zmax, xmin, ymin, zmin))



# Interface with Wannier90

In [ ]:
def create_siesta2wannier90_runfile(H, name:str, tot_num_bands=None, path=None,
    spin_polarized=False, spin_afm=True):

    run_file = name + '_S2WRUN.fdf'
    struct_file = name + '_STRUCT.fdf'
    
    cell = H.cell
    a, b, c = cell.diagonal()
    
    if not tot_num_bands:
        tot_num_bands = len(np.where(H.eigh()<0)[0])

    if not path:
        path = './s2w/vb'
    with open(os.path.join(path,f'{run_file}'), 'w') as fout: #generating input file
        fout.write(f"""%include {struct_file}
SystemName              {name}
SystemLabel             {name}

AtomCoorFormatOut Ang
###########################################
#                 PAO
###########################################
PAO.BasisSizes          DZP
PAO.BasisType           split
PAO.EnergyShift         150 meV
%block kgrid_Monkhorst_Pack
   20     0     0   0.0 #converge
   0     1     0   0.0
   0     0     1   0.0
%endblock kgrid_Monkhorst_Pack
###########################################
#  Molecular Dynamics
###########################################
MD.VariableCell         T
MD.TypeOfRun            CG
MD.NumCGsteps           1000
MD.MaxCGDispl           0.05  Ang
MD.MaxForceTol          0.01 eV/Ang
MD.UseSaveXV            F
###########################################
#      Density Matrix
###########################################
DM.UseSaveDM            T
MaxSCFIterations        500
DM.MixingWeight         0.05
DM.NumberPulay          4
DM.Tolerance            1.d-4
###########################################
#    other parameters
###########################################
SolutionMethod          diagon
Diag.DivideAndConquer   False
MeshCutoff              400 Ry #converge
xc.functional           GGA #functional LDA vs GGA
xc.authors              PBE #functional PBE vs PBEsol etc.
ElectronicTemperature   300 K
# ###########################################
# #     Band Structures
# ###########################################
BandLinesScale  pi/a
%block BandLines
  1         0.00000         0.00000         0.00000         \Gamma  
 100        1.00000         0.00000         0.00000         \X 
%endblock BandLines

###########################################
#Output parameters switches
###########################################

WriteMullikenPop        1
SaveRho                 T
WriteVoronoiPop         T
WriteEigenvalues        T
UseSaveData             T
SaveHS                  T # save hamiltonian
WriteCoorXmol           T # final coordinates file
WriteCoorStep           T # write coordinates to .XV files and overwrite at each step
WriteMDXmol             T
NetCharge               0

############################################
# Interface with Wannier90
############################################

Siesta2Wannier90.WriteMmn     true
Siesta2Wannier90.WriteAmn     true
Siesta2Wannier90.WriteEig     true
Siesta2Wannier90.WriteUnk     true

Siesta2Wannier90.NumberOfBands {tot_num_bands}

Siesta2Wannier90.UnkGridBinary true
Siesta2Wannier90.UnkGrid1     20
Siesta2Wannier90.UnkGrid2     20
Siesta2Wannier90.UnkGrid3     20
""")
        if spin_polarized:
            fout.write(f"""
##############################################
# Spin Settings
##############################################
Spin                    polarized
# this can only change the initial spin state, if read from DM, this parameter
# is useless. So, delete .DM file before change this parameter
DM.InitSpin.AF          {spin_afm}
""")


In [ ]:
def create_win(geom, name, path=None, tot_num_bands=None,
               num_ex_bands=None, num_wann=None,
               kmesh=[12,1,1],dis_win_max=None,
               dis_win_min=None, dis_froz_max=None,
               dis_froz_min=None, guiding_centres=False):
    # by default we use [12, 1, 1] as kmesh
    # by default we exclude all the s bands, this already gives very good result
    # the default values only work for DZP basis set!!!

    C_atoms = []
    for i, at in enumerate(geom.atoms):
        if at.Z == 6:
            C_atoms.append(i)
    if not num_wann:
        # this only works for DZP basis set
        num_wann = int(len(C_atoms)/2)
    if not num_ex_bands:
        # this only works for DZP basis set
        num_ex_bands = int(len(C_atoms)/2)
    num_bands = tot_num_bands - num_ex_bands
    
    # proj_orb = ''
    # for i in range(num_wann):
    #     proj_orb += f'{2*i+1} '
    proj_orb = f'1-{num_wann}'        
    
    if not dis_win_min:
        dis_win_min = -30
    if not dis_win_max:
        dis_win_max = 0
    
    if not path:
        path = './s2w/vb'
    with open(os.path.join(path, f"{name}.win"), 'w') as fwin:
        fwin.write(f"""
num_bands   =   {num_bands}
num_wann    =   {num_wann}
""")
        if num_ex_bands:
            fwin.write(f"exclude_bands =  1-{num_ex_bands}\n")
        if dis_win_min:
            fwin.write(f"dis_win_min =  {dis_win_min}\n")
        if dis_win_max:
            fwin.write(f"dis_win_max =  {dis_win_max}\n")
        if dis_froz_min:
            fwin.write(f"dis_froz_min =  {dis_froz_min}\n")
        if dis_froz_max is not None:
            fwin.write(f"dis_froz_max =  {dis_froz_max}\n")

        fwin.write(f"""
begin projections
C: l=1, mr=1
N: l=1, mr=1
Cu: l=2
end projections

select_projections: {proj_orb}

iprint : 3

num_iter	=	500
write_hr	=	true
write_tb	=	true
write_xyz   =   true
translate_home_cell =   true
!translation_centre_frac
guiding_centres =  {str(guiding_centres)}
!trial_step  =   1.0

!bands_plot      =   true
wannier_plot    =   true
wannier_plot_supercell  =  3, 1, 1
!wannier_plot_mode = molecule
kmesh_tol = 0.0001

""")
        # write unit cell cart
        fwin.write("begin unit_cell_cart\n")
        fwin.write("Ang\n")
        for i in range(len(geom.cell)):
            fwin.write("  {:.10f}\t{:.10f}\t{:.10f}\n".format(*geom.cell[i]))
        fwin.write("end unit_cell_cart\n\n")
        fwin.write("""
begin kpoint_path                         
G 0.000  0.000  0.000 X 0.500  0.000  0.000 
X 0.500  0.000  0.000 G 1.000  0.000  0.000 
end kpoint_path

""")
        fwin.write("mp_grid: {} {} {}\n".format(*kmesh))
        fwin.write("begin kpoints\n")
        for i in range(kmesh[0]):
            fwin.write("\t{:.8f}\t{:.8f}\t{:.8f}\n".format(i/kmesh[0], 0, 0))
        fwin.write("end kpoints\n\n")

        # write atom coordinates
        fwin.write("begin atoms_cart\n")
        fwin.write("Ang\n")
        for i in range(len(geom.xyz)):
            fwin.write("{}\t{:.10f}\t{:.10f}\t{:.10f}\n".format(
                geom.atoms[i].symbol, *geom.xyz[i]
            ))
        fwin.write("end atoms_cart\n\n")


In [ ]:
###customise RUN.fdf File here. See SIESTA docs for reference.

def create_wannier90insiesta_runfile(H, name:str):

    run_file = name + '_WinSRUN.fdf'
    struct_file = name + '_WinSSTRUCT.fdf'
    
    cell = H.cell
    a, b, c = cell.diagonal()
    
    tot_num_bands = len(np.where(H.eigh()<0)[0])

    with open(f'./WinS/{run_file}', 'w') as fout: #generating input file
        fout.write(f"""%include {struct_file}
SystemName              {name}
SystemLabel             {name}

AtomCoorFormatOut Ang
###########################################
#                 PAO
###########################################
%block PAO.BasisSizes
   C    DZP
   H    DZP
%endblock PAO.BasisSizes
PAO.BasisType           split
PAO.EnergyShift         150 meV
%block kgrid_Monkhorst_Pack
   20     0     0   0.0 #converge
   0     1     0   0.0
   0     0     1   0.0
%endblock kgrid_Monkhorst_Pack
###########################################
#  Molecular Dynamics
###########################################
MD.VariableCell         T
MD.TypeOfRun            CG
MD.NumCGsteps           1000
MD.MaxCGDispl           0.05  Ang
MD.MaxForceTol          0.01 eV/Ang
MD.UseSaveXV            F
###########################################
#      Density Matrix
###########################################
DM.UseSaveDM            T
MaxSCFIterations        500
DM.MixingWeight         0.05
DM.NumberPulay          4
DM.Tolerance            1.d-4
###########################################
#    other parameters
###########################################
SolutionMethod          diagon
MeshCutoff              400 Ry #converge
xc.functional           GGA #functional LDA vs GGA
xc.authors              PBE #functional PBE vs PBEsol etc.
ElectronicTemperature   300 K
# ###########################################
# #     Band Structures
# ###########################################
BandLinesScale  pi/a
%block BandLines
  1         0.00000         0.00000         0.00000         \Gamma  
 400        2.00000         0.00000         0.00000         \Gamma  
%endblock BandLines

###########################################
#Output parameters switches
###########################################

WriteMullikenPop        1
SaveRho                 T
WriteVoronoiPop         T
WriteEigenvalues        T
UseSaveData             T
SaveHS                  T # save hamiltonian
WriteCoorXmol           T # final coordinates file
WriteCoorStep           T # write coordinates to .XV files and overwrite at each step
WriteMDXmol             T
NetCharge               0

############################################
# Interface with Wannier90
############################################

NumberOfBandManifoldsForWannier   1

%block WannierManifolds
  1                 # Sequential index of the manifold, from 1 to NumberOfBandManifoldsForWannier
  13   26              # Indices of the initial and final band of the manifold
  6                # Number of bands for Wannier transformation
  4  17  30  43  56  69 # 82 95 108 121 134 147 160 173        # Indices of the orbitals that will be used as localized trial orbitals
  num_iter 500         # Number of iterations for the minimization of \Omega
  wannier_plot  3    # Plot the Wannier function 
  fermi_surface_plot False # Plot the Fermi surface
  write_hr           # Write the Hamiltonian in the WF basis
  write_tb           # Write the Hamiltonian in the WF basis
  -30.0    -0       # Bottom and top of the outer energy window for band disentanglement (in eV)
  -30.0    -20       # Bottom and top of the inner energy window for band disentanglement (in eV)
%endblock WannierManifolds

%block kMeshforWannier
   12  1  1  
%endblock kMeshforWannier

Siesta2Wannier90.UnkGrid1       30
Siesta2Wannier90.UnkGrid2       30
Siesta2Wannier90.UnkGrid3       30

Wannier90_in_SIESTA_compute_unk .true.
""")


# Phonon Calculation

In [ ]:
###customise RUN.fdf File here. See SIESTA docs for reference.

def create_fcbuild_file(name:str, geom: Geometry, supercell=[1,0,0], path='./phonon',
      mesh_cuttoff=400):

   fcbuild_file = name + '.fcbuild.fdf'
   # commonly used atoms
   mass_list = {'H': 1.0079, 'C': 12.0107, 
                'N': 14.0067, 'O': 15.9994, 
                'Cl': 35.4530, 'V': 50.9415, 
                'Cu': 63.5460, 'Zn': 65.3900,
                'Br': 79.9040}
   # for a in geom.atoms.atom:
   #    a.mass = mass_list[a.tag]
   
   sc1, sc2, sc3 = supercell
   lc = np.linalg.norm(geom.cell[0])
   cellfrac = geom.cell/lc

   with open(os.path.join(path, f'{fcbuild_file}'), 'w') as fcb: 
      fcb.write(f"""
SystemName           {name}
SystemLabel          {name}

NumberOfSpecies      {geom.atoms.nspecie}
NumberOfAtoms        {geom.na}

PAO.BasisSizes       DZP
Eigenvectors         .true.
%block kgrid_Monkhorst_Pack
   20    0     0   0.0 
   0     1     0   0.0
   0     0     1   0.0
%endblock kgrid_Monkhorst_Pack
MeshCutoff           {mesh_cuttoff} Ry

SuperCell_1          {sc1} 
SuperCell_2          {sc2}    
SuperCell_3          {sc3} 

BandLinesScale       pi/a
%block BandLines
  1         0.00000         0.00000         0.00000         Gamma
 100        1.00000         0.00000         0.00000         X
%endblock BandLines

%block ChemicalSpeciesLabel""")
      for i, a in enumerate(geom.atoms.atom):
         fcb.write(f"\n{i+1}\t{a.Z}\t{a.symbol}")
      fcb.write(f"""
%endblock ChemicalSpeciesLabel

LatticeConstant      {lc} Ang
%block LatticeVectors""")
      for i in range(len(geom.cell)):
         fcb.write("\n{:.8f}\t{:.8f}\t{:.8f}".format(*cellfrac[i]))
      fcb.write("""
%endblock LatticeVectors

AtomicCoordinatesFormat NotScaledCartesianAng
%block AtomicCoordinatesAndAtomicSpecies""")
      for ia, a, isp in geom.iter_species():
         fcb.write("\n{:.8f}\t{:.8f}\t{:.8f}\t{}  {}\t# {} {}".format(
            *geom.xyz[ia,:], isp+1, mass_list[a.tag], ia+1, a.tag))
      fcb.write(f"""
%endblock AtomicCoordinatesAndAtomicSpecies
""")


In [ ]:
###customise RUN.fdf File here. See SIESTA docs for reference.

def create_ifc_file(name:str, geom: Geometry, path='./phonon',
      mesh_cuttoff=400):

   ifc_file = name + '.ifc.fdf'
   if not path:
      path = './phonon'
   with open(os.path.join(path,f'{ifc_file}'), 'w') as fcb: 
      fcb.write(f"""
SystemName           {name}
SystemLabel          {name}

NumberOfSpecies      {geom.atoms.nspecie}
NumberOfAtoms        < FC.fdf

PAO.BasisSizes       DZP

%block kgrid_Monkhorst_Pack
   20    0     0   0.0 
   0     1     0   0.0
   0     0     1   0.0
%endblock kgrid_Monkhorst_Pack

MeshCutoff           {mesh_cuttoff} Ry

%block ChemicalSpeciesLabel""")
      for i, a in enumerate(geom.atoms.atom):
         fcb.write(f"\n{i+1}\t{a.Z}\t{a.symbol}")
      fcb.write("""
%endblock ChemicalSpeciesLabel

LatticeConstant      < FC.fdf
LatticeVectors       < FC.fdf
AtomicCoordinatesFormat             < FC.fdf     
AtomicCoordinatesAndAtomicSpecies   < FC.fdf

MD.TypeOfRun    < FC.fdf   # Compute the interatomic force constants matrix 
MD.FCfirst      < FC.fdf   # Index of first atom to displace
MD.FClast       < FC.fdf   # Index of the last atom to displace
MD.FCdispl      < FC.fdf   # Displacement to use for the computation of the
                           #   interatomic force constant matrix
                           #   (Remember that the second derivative of the 
                           #   energy with respect the displacement of two
                           #   atoms is computed by means of a 
                           #   finite difference derivative of the forces)
""")


In [ ]:
def read_phonon_bands(name, path=None, as_dataarray=True, squeeze=True):

    if not path:
        path = './phonon'
    bands_path = os.path.join(path, f'{name}.bands')

    bandsile = get_sile(bands_path)
    bands = bandsile.read_data(as_dataarray=as_dataarray)
    if squeeze:
        bands = bands.squeeze()
    if bands.ticklabels[0] == 'Gamma':
        bands.ticklabels[0] = '$\Gamma$'
    if bands.ticklabels[1] == 'X':
        bands.ticklabels[1] = '$X$'
    return bands

In [ ]:
def plot_phonon_bands(bands: xarray.DataArray,name, E_range=None, figsize=(8,6),
                     ticks_font=12, label_font=12, title_font=14, border_line_width=2, 
                     save=False, path=None, save_format='png', dpi=600, ax2=True, **kwargs):
    
    if E_range:
        enrg_range = np.array(E_range)
        freq_range = enrg_range*8.066

    ks = bands.k.data
    fig = plt.figure(figsize=figsize)
    ax1 = fig.add_subplot(111)
    if E_range:
        for i in range(bands.shape[1]):
            if np.any(np.logical_and(bands[:,i]>E_range[0]*8.066,
                    bands[:,i]<E_range[1]*8.066)):
                ax1.plot(ks, bands[:,i]/8.066, color='k', **kwargs)
    else:
        for i in range(bands.shape[1]):
            ax1.plot(ks, bands[:,i]/8.066, color='k', **kwargs)
        
    ax1.set_xticks(bands.ticks)
    ax1.set_xticklabels(bands.ticklabels, fontsize=ticks_font)
    ax1.set_xlim(bands.ticks)
    ax1.set_ylabel('Energy (meV)', fontsize=label_font)
    fig_title = '{}_PhononDispersion'.format(name)
    ax1.set_title(fig_title+'\n', fontsize=title_font)

    for border in ['left', 'bottom', 'top', 'right']:
        ax1.spines[border].set_linewidth(border_line_width)

    if ax2:
        ax2 = ax1.twinx()
        ax2.set_ylabel('Frequency ($cm^{-1}$)', fontsize=label_font)
        ax2.set_ylim(np.array(ax1.get_ylim())*8.066)

    if E_range:
        ax1.set_ylim(enrg_range)
        if ax2:
            ax2.set_ylim(freq_range)

    if save:
        if E_range:
            fig_name = name + '_{}to{}meV'.format(*enrg_range)
        else:
            fig_name = name + '_full_range'
        fig_path = os.path.join(path, fig_name+'.'+save_format)
        fig.savefig(fig_path, dpi=dpi)


In [ ]:
def write_GammaPhonon_xsf(name, geom, first_band: int, last_band: int,
        path=None, num_of_cells=1):
    if not path:
        path = './phonon'
    vec_path = os.path.join(path, f'{name}.vectors')
    with open(vec_path, 'r') as vf:
        for l in range(2):
            vf.readline()
        for b in range(first_band-1):
            for l in range(2+2*(geom.na+1)):
                vf.readline()
        for b in np.arange(first_band, last_band+1, 1):
            band = int(vf.readline()[-6:].strip()) # label of band
            freq = float(vf.readline()[14:].strip()) # cm-1
            enrg = freq/8.066
            vf.readline() # real part
            vector = np.empty((geom.na, 3))
            for i in range(geom.na):
                v = vf.readline().strip().split()
                vec = np.array([float(a) for a in v])
                vector[i,:] = vec
            for i in range(geom.na+1):
                vf.readline()

            with open(os.path.join(path, 
                '{}_GammaMode_{}_{:.2f}meV.xsf'.format(name, band, enrg)), 'w') as xsf:
                xsf.write("# ---- XSF block for ---- \n")
                xsf.write(f"# mode = {band} (Gamma)\n")
                xsf.write(f"# frequency = {freq} cm-1\n")
                xsf.write(f"# energy = {enrg} meV\n")
                xsf.write('CRYSTAL\n')
                xsf.write('PRIMVEC\n')
                for i in range(3):
                    xsf.write('  {:.8f}  {:.8f}  {:.8f}\n'.format(*geom.cell[i,:]))
                xsf.write('CONVVEC\n')
                for i in range(3):
                    xsf.write('  {:.8f}  {:.8f}  {:.8f}\n'.format(*geom.cell[i,:]))
                xsf.write('PRIMCOORD\n')
                xsf.write('  {}  {}\n'.format(geom.na, num_of_cells))
        
                for i, a, _ in geom.iter_species():
                    xsf.write('  {}  {:.8f}  {:.8f}  {:.8f}  {:.8f}  {:.8f}  {:.8f}\n'.format(
                        a.Z, *geom.xyz[i,:], *vector[i,:]))

